In [1]:
import sys
import re
import numpy as np
from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.ml.linalg import Vectors, VectorUDT
from operator import add

In [2]:
df = spark.read.option('header',True).csv('news.csv')\
    .drop('Title')


In [3]:
pn_udf = udf(lambda x: int(x), IntegerType() )
dfl = df.withColumn('label',pn_udf('Class')).drop('Class')

In [4]:
df_l = dfl.selectExpr("label as label","Description as text")

In [5]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer
dim = 20000
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'words')
wordsData = tokenizer.transform(df_l)

In [6]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=dim)
featurizedData = hashingTF.transform(wordsData)

In [7]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

In [8]:
rescaledData = idfModel.transform(featurizedData)
tfifd = rescaledData.select("label", "features")

In [9]:
df_train, df_test = tfifd.randomSplit([.7, .3])

In [10]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=100)
model = lr.fit(df_train)
predictions = model.transform(df_test)

In [11]:
result = predictions.select('label','prediction').collect()
y = []
y_pred = []

for i in result:
    y.append(i[0])
    y_pred.append(i[1])

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
conf = confusion_matrix(y, y_pred)
print('confusion matrix is ')
print(conf)
print('accuracy is:')
print(accuracy_score(y, y_pred))
print('f1 score is:')
print(classification_report(y, y_pred))

confusion matrix is 
[[7314  372  629  595]
 [ 394 8083  217  261]
 [ 593  149 7051 1136]
 [ 572  202 1278 7063]]
accuracy is:
0.8218273970313849
f1 score is:
              precision    recall  f1-score   support

           1       0.82      0.82      0.82      8910
           2       0.92      0.90      0.91      8955
           3       0.77      0.79      0.78      8929
           4       0.78      0.77      0.78      9115

    accuracy                           0.82     35909
   macro avg       0.82      0.82      0.82     35909
weighted avg       0.82      0.82      0.82     35909

